# Laboratoire 4 - Cinématique

> **Attention!** Il est *strictement interdit* de s'approcher des commandes qui font avancer le robot si il y a le moindre risque que le robot ne tombe par terre. Les équipes qui ont un accidents de robot devront chanter *Never Gonna Give You Up* devant toute la classe.

Au cours de ce laboratoire nous allons étudier la façon dont le robot se déplace. Comme vous le constaterez, le sujet est plus complexe qu'il n'y paraît! Connectons-nous d'abord au robot.


In [1]:
from robmob import robot, sensors

robot_ip = '192.168.0.105'
robot = robot.Robot(robot_ip)
robot.connect()

## Partie 1 - Le problème du contrôle

Pour cette première expérience, utilisez les fonctions de la librairie `robmob` pour ordonner au robot d'avancer de un mètre.

In [3]:
robot.linear_movement(0.2, 5)  # Ordonne au robot d'avancer à 0.2 m/s pendant 5 s

Vous constaterez que les kobukis se déplacent de beaucoup moins qu'un mètre avec l'exécution de cette command. Qu'est-ce qui se passe? Pour le comprendre, il faut d'abord se demander comment le robot exécute notre demande.

Nous envoyons une commande de vitesse au robot. Le robot active ses moteurs, puis après un certain $\Delta t$ le robot consulte les encodeurs de ses roues pour valider sa vitesse. La différence entre la vitesse demandée et la vitesse réelle constitue une erreur. Cette erreur est fournie à une fonction appelée un [correcteur PID](https://fr.wikipedia.org/wiki/R%C3%A9gulateur_PID) qui traduit l'erreur en une nouvelle consigne. Cette nouvelle consigne tente de corriger l'erreur sur la vitesse du robot, pour exécuter aussi bien que possible la commande qu'on lui a demandé.

Malheureusement, il est difficile de créer un correcteur parfait. Dans le cas qui nous concerne, le correcteur semble avoir de la difficulté à atteindre la vitesse demandée, surtout au début de la trajectoire. Voici un graphique de la vitesse instantanée d'un robot kobuki par rapport au temps. La vitesse demandée est en rouge. L'apparence de ce graphique explique pourquoi les distances parcourues par le robot sont plus courtes que les distances demandées.

![Vitesse de la kobuki par rapport au temps](img/vitesse_kobuki.png)



In [ ]:
robot.general_movement_command(0.2, 0, 10)

In [ ]:
MAX_LINEAR_ACCELERATION = 0.5
MIN_LINEAR_ACCELERATION = -0.5 * 1.2

MAX_ANGULAR_ACCELERATION = 3.5
MIN_ANGULAR_ACCELERATION = -3.5 * 1.2

def time_of_linear_movement(speed, distance):
    total_time = 0.
    
    time_to_top_speed = speed / MAX_LINEAR_ACCELERATION
    distance -= time_to_top_speed * speed / 2.
    total_time += time_to_top_speed
    
    time_to_stop = -speed / MIN_LINEAR_ACCELERATION
    distance -= time_to_stop * speed / 2.
    total_time += time_to_stop
    
    total_time += distance / speed
    
    return total_time

def time_of_angular_movement(speed, angle):
    total_time = 0.
    
    time_to_top_speed = speed / MAX_ANGULAR_ACCELERATION
    angle -= time_to_top_speed * speed / 2.
    total_time += time_to_top_speed
    
    time_to_stop = -speed / MIN_ANGULAR_ACCELERATION
    angle -= time_to_stop * speed / 2.
    total_time += time_to_stop
    
    total_time += angle / speed
    
    return total_time
    

In [ ]:
time_of_linear_movement(0.2, 1)
time_of_angular_movement(3.1416/5, 3.1416)

In [ ]:
5/5.3

## Grapher les ticks de l'odomètre

In [ ]:
odometer = sensors.OdometerTicksSensor(30 * sensors.OdometerTicksSensor.SAMPLE_RATE) # 10 sec buffer size.
robot.add_sensor(odometer)

In [ ]:
import numpy as np

def get_odometry_data():
    odometry_data = []
    buf_snapshot = odometer.read_buffer()
    for data in buf_snapshot:
        row = [
                data[0]['secs'] * 1e9 + data[0]['nsecs'],
                data[1],
                data[2]
            ]
        odometry_data.append(row)

    odometry_data = np.array(odometry_data)
    odometry_data[:,0] -= odometry_data[0,0]
    return odometry_data

def average_tps_of_odom_data(odom_data):
    time_of_min = 0.0
    
    i = 0
    while odom_data[i,1] == odom_data[0,1]:
        i += 1
    
    time_of_min = odom_data[i,0]
    value_of_min = odom_data[i,1]
    
    while odom_data[i,1] < odom_data[i + 1, 1]:
        i += 1
    
    time_of_max = odom_data[i+1, 0]
    value_of_max = odom_data[i+1, 1]
    
    print('max {}, min {}'.format(value_of_max, value_of_min))
    
    return (value_of_max - value_of_min) / (time_of_max - time_of_min) * 1e9

In [ ]:
import time

robot.move(0.2, time_of_linear_movement(0.2, 3))
time.sleep(2)
odometry_data = get_odometry_data()

odometry_data

In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time

In [ ]:


plt.plot(odometry_data[:, 0], odometry_data[:, 1], 'b')
plt.plot(odometry_data[:, 0], odometry_data[:, 2], 'r')
plt.show()

In [ ]:
deltas = []

for i in range(1, odometry_data.shape[0] - 1):
    n_ticks = odometry_data[i + 1,1] - odometry_data[i - 1,1]
    
    if n_ticks < 0:
        continue
    
    delta_t = (odometry_data[i+1, 0] - odometry_data[i - 1,0]) / 1e9
    
    delta_x = n_ticks / 11724.41658029856624751591
    
    
    deltas.append([odometry_data[i,0], delta_x / delta_t])
    
deltas = np.array(deltas)

plt.axhline(0.2, linestyle='--', color='r')
plt.plot(deltas[:,0], deltas[:,1])
plt.xlabel('Temps (s)')
plt.ylabel('Vitesse (m/s)')


In [ ]:
import time

robot.move(0.2, time_of_linear_movement(0.2, 2))
time.sleep(2)
odometry_data = get_odometry_data()

In [ ]:
average_tps_of_odom_data(odometry_data)

In [ ]:
calibration_data = []

for distance in [0.2, 0.4, 0.6, 0.8, 1.0]:
    robot.move(0.2, time_of_linear_movement(0.2, distance))
    time.sleep(2)
    odometry_data = get_odometry_data()
    calibration_data.append([distance, average_tps_of_odom_data(odometry_data)])
    
calibration_data = np.array(calibration_data)

In [ ]:
odometry_data

In [ ]:
average_tps_of_odom_data(odometry_data)

In [ ]:
calibration_data

In [ ]:
plt.plot(calibration_data[:,0], calibration_data[:, 1])
plt.show()

In [ ]:
m = (calibration_data[4, 1] - calibration_data[1,1]) / (calibration_data[4,0] - calibration_data[1,0])

In [ ]:
m

In [ ]:
calibration_data[4,1]

In [ ]:
b = calibration_data[4,1] - m*calibration_data[4,0]

In [ ]:
b

In [ ]:
m = m / 11724

In [ ]:
b = b / 11724

In [ ]:
m

In [ ]:
b

In [ ]:
m + b

In [ ]:
0.2 / (m + b)

In [ ]:
0.2 * 1.18